# Chapter 54: Agentic Workflow Design Patterns
Orchestrator-worker, tool selection, and state machines.


In [ ]:
import json
print("Agentic Patterns ready")


In [ ]:
class Worker:
    def __init__(self,name,caps): self.name=name; self.caps=caps; self.done=0
    def can(self,t): return t in self.caps
    def execute(self,task): self.done+=1; return self.name+" completed "+task["type"]
class Orchestrator:
    def __init__(self): self.workers=[]
    def register(self,w): self.workers.append(w)
    def dispatch(self,task):
        el=[w for w in self.workers if w.can(task["type"])]
        if not el: return "No worker for "+task["type"]
        return min(el,key=lambda x:x.done).execute(task)
orch=Orchestrator()
orch.register(Worker("network-agent",["scan","diagnose","monitor"]))
orch.register(Worker("security-agent",["threat_hunt","scan","alert"]))
orch.register(Worker("remediation-agent",["patch","isolate","rollback"]))
for task in [{"type":"scan"},{"type":"threat_hunt"},{"type":"patch"},{"type":"diagnose"}]:
    print("["+task["type"]+"]:",orch.dispatch(task))


In [ ]:
TOOLS={"web_search":{"cost":1,"for":["lookup","research"]},"db_query":{"cost":2,"for":["data","history"]},"api_call":{"cost":3,"for":["live","realtime"]},"local_kb":{"cost":0,"for":["policy","config"]}}
def select_tool(query,budget=5):
    words=query.lower().split()
    scores={t:sum(1 for w in words if any(w in tag for tag in s["for"]))-s["cost"]*0.1 for t,s in TOOLS.items() if s["cost"]<=budget}
    return max(scores,key=scores.get) if scores else None
for q in ["Get current BGP session status","Look up VLAN policy","Find historical log data"]:
    print(q[:50],"->",select_tool(q))
state="idle"; history=["idle"]
TRANS={"idle":{"start":"planning"},"planning":{"ready":"executing","fail":"idle"},"executing":{"done":"reporting"}}
def transition(ev):
    global state
    nxt=TRANS.get(state,{}).get(ev)
    if nxt: state=nxt; history.append(state); return True
    return False
for ev in ["start","ready","done"]: transition(ev)
print("State path:"," -> ".join(history))
